In [1]:
from google import genai
import os
from dotenv import load_dotenv
import duckdb
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# response = client.models.generate_content(
#     model="gemini-2.0-flash", contents="Ge mig en historia om en kanin som bor i skogen på 3 meningar"
# )
# print(response.text)



In [2]:
with duckdb.connect("../../ads_data.duckdb") as con: 
    df_mart = con.execute("SELECT * FROM mart.mart_kultur_media").df()
df_test = df_mart #[df_mart["occupation_field"] == "Kultur, media, design"]
# df_test.to_csv("sample_ads.csv") # måste göra mer finjusteringar! kan inte läsa allas description direkt!


In [4]:
descriptions = df_test["description"]

top_employer = df_test.groupby("employer_name")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()
top_employer
# df_test

,employer_name,vacancies,description
0,Vs Consulting AB,500,Vi är en modern tolkförmedling som bokar och f...
1,Exakta Photo AB,100,Exakta Photo är en av Sveriges största skolfot...
2,Stegra Boden AB,74,Research shows that women and other under-repr...
3,Språkservice Sverige AB,64,Språkservice Sverige AB är Sveriges största to...
4,H & M Hennes & Mauritz Gbc AB,48,WHAT YOU’LL DO​ \nAs a Global visual & commerc...
...,...,...,...
229,Zinzino Operations AB,1,Zinzino is a global direct sales company from ...
230,ÖSTRA GÖINGE KOMMUN,1,Östra Göinge kommun med drygt 14 000 invånare ...
231,Örebro Universitet,1,Örebro universitet skapar framtiden genom inte...
232,Öregrund Marine Service AB,1,Har du erfarenhet inom eftermarknad och/eller ...


In [116]:
nr_1 = top_employer[top_employer["employer_name"] == 'Exakta Photo AB']
nr_1["description"].to_csv("top_employer.csv")
# nr_1["description"]

In [6]:
df_test.reset_index()

,index,headline,occupation,occupation_group,occupation_field,description,duration,working_hours_type,scope_of_work_min,scope_of_work_max,vacancies,employer_name,workplace_region,workplace_city,experience_required,driving_license,access_to_own_car,must_have_languages
0,0,AV-tekniker till Mediainstallation Sverige AB,AV-tekniker,"Ljus-, ljud- och scentekniker","Kultur, media, design",AV-tekniker till Mediainstallation AB \nVi väx...,Tills vidare,Heltid,100,100,1,Mediainstallation i Sverige AB,Stockholms län,Täby,True,True,False,ej specifierat
1,1,Logistikkoordinator på heltid,"Produktionsassistent: kultur, media, film, dat...",Inspicienter och scriptor m.fl.,"Kultur, media, design",Vi söker dig som vill bli spindeln i nätet på ...,6 månader eller längre,Heltid,100,100,1,Academic Work Sweden AB,Stockholms län,Stockholm,True,False,False,ej specifierat
2,2,Grafiker | Lernia | Norrköping,Grafisk formgivare,Grafiska formgivare m.fl.,"Kultur, media, design",Är du en kreativ grafiker med erfarenhet av In...,11 dagar - upp till 3 månader,Deltid,0,100,1,Lernia Bemanning AB,Östergötlands län,Norrköping,True,False,False,ej specifierat
3,3,Global Field Operation Manager,Technical Operation Manager/TOM,Bild- och sändningstekniker,"Kultur, media, design",The Opportunity:\nWith a pioneering technology...,Tills vidare,Heltid,100,100,1,Hitachi Energy Sweden AB,Västmanlands län,Västerås,True,False,False,ej specifierat
4,4,Tekniska museet söker en Föremålsantikvarie,Antikvarie,Museiintendenter m.fl.,"Kultur, media, design",Tekniska museet söker en föremålsantikvarie ti...,Tills vidare,Heltid,100,100,1,Stiftelsen Tekniska Museet,Stockholms län,Stockholm,True,True,False,ej specifierat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,680,Tolk i Singalesiska,Tolk,"Översättare, tolkar och lingvister m.fl.","Kultur, media, design",Talar du flytande Singalesiska och svenska? Sö...,Tills vidare,Heltid,100,100,1,Språkservice Sverige AB,ej angiven,ej angiven,False,False,False,ej specifierat
681,681,Tolk i Singalesiska,Tolk,"Översättare, tolkar och lingvister m.fl.","Kultur, media, design",Talar du flytande Singalesiska och svenska? Sö...,Tills vidare,Heltid,100,100,1,Språkservice Sverige AB,ej angiven,ej angiven,False,False,False,ej specifierat
682,682,Tolk i Singalesiska,Tolk,"Översättare, tolkar och lingvister m.fl.","Kultur, media, design",Talar du flytande Singalesiska och svenska? Sö...,Tills vidare,Heltid,100,100,1,Språkservice Sverige AB,ej angiven,ej angiven,False,False,False,ej specifierat
683,683,Tolk i Singalesiska,Tolk,"Översättare, tolkar och lingvister m.fl.","Kultur, media, design",Talar du flytande Singalesiska och svenska? Sö...,Tills vidare,Heltid,100,100,1,Språkservice Sverige AB,ej angiven,ej angiven,False,False,False,ej specifierat


In [117]:
with open("top_employer.csv", "r", encoding='utf-8') as file: 
    ad_text = file.read() 
# ad_text = nr_1["description"]

promt = f"""Du är en rekryterare inom data Media, Kultur, Design.
Plocka ut 5 av de vanligaste kraven de 5 vanligaste meriterade och antal av dessa som efterfrågas i dessa jobbannonser:
{ad_text}
Output ska vara i detta formatet enbart, utan kommentarer från dig: 

{{
    "krav": [erfarenhet1, erfarenhet2, ...]
    "antal krav": [summa erfarenhet1..]
    "meriterande": [meriterande1,meriterande2, ... ]
    "antal meriterande": [summa meriterande1..]
    
}}

"""

In [118]:
response = client.models.generate_content(
    model="gemini-2.0-flash", contents=promt
)
print(response.text)

```json
{
    "krav": ["Fotointresse", "Körkort och tillgång till bil", "God datorvana", "Utåtriktad och positiv", "Stresstålig och välorganiserad"],
    "antal krav": [20, 14, 14, 7, 4],
    "meriterande": ["Erfarenhet av barn- och ungdomsverksamhet", "Tidigare branscherfarenhet inom foto/skolfoto", "Vana av Adobes program", "Etablerad fotograf", "Social förmåga och estetisk blick"],
    "antal meriterande": [8, 7, 4, 3, 2]
}
```


In [119]:
# print(repr(response.text))
data = response.text
cleaned = data.strip("```json") #.replace("\n", "").strip().replace("  ", "")
cleaned
# data

'\n{\n    "krav": ["Fotointresse", "Körkort och tillgång till bil", "God datorvana", "Utåtriktad och positiv", "Stresstålig och välorganiserad"],\n    "antal krav": [20, 14, 14, 7, 4],\n    "meriterande": ["Erfarenhet av barn- och ungdomsverksamhet", "Tidigare branscherfarenhet inom foto/skolfoto", "Vana av Adobes program", "Etablerad fotograf", "Social förmåga och estetisk blick"],\n    "antal meriterande": [8, 7, 4, 3, 2]\n}\n'

In [120]:
type(cleaned)
import json
import pandas as pd
data = json.loads(cleaned)
df = pd.DataFrame({
    "krav": data["krav"], 
    "antal krav": data["antal krav"],
    "meriterande": data["meriterande"],
    "antal meriterande": data["antal meriterande"]
    
})
df
# df = pd.DataFrame(data)
# df
# df = pd.DataFrame(df)
# df

,krav,antal krav,meriterande,antal meriterande
0,Fotointresse,20,Erfarenhet av barn- och ungdomsverksamhet,8
1,Körkort och tillgång till bil,14,Tidigare branscherfarenhet inom foto/skolfoto,7
2,God datorvana,14,Vana av Adobes program,4
3,Utåtriktad och positiv,7,Etablerad fotograf,3
4,Stresstålig och välorganiserad,4,Social förmåga och estetisk blick,2


In [123]:
import plotly.express as px
# px.pie(df, names=["krav", "meriterande"], values=["antal krav", "antal meriterande"])
px.bar(df, x="krav", y="antal krav")


In [124]:
px.bar(df, y="antal meriterande", x="meriterande")